<a href="https://colab.research.google.com/github/ce888-sr/CE888/blob/master/Lab4/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [227]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [228]:
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv', header=None)
test = df
test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.0,99.00,99.0,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.0,99.00,99.0,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.0,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.0,99.00,99.0,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [229]:
#replace the value 99 with NaN
test = test.replace(99, np.nan).copy()

In [230]:
#drop column 1 as it contains jokes not ratings
test = test.drop([0], axis=1).copy()

In [231]:
test

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.42,NaN,NaN,NaN,-7.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,7.91,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,NaN,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,NaN,8.25,NaN,NaN,7.48,7.28,7.28,8.93,NaN,6.17,7.28,NaN,NaN,8.98,7.33,NaN,6.17,9.08,7.33,...,6.46,7.28,NaN,NaN,7.04,7.28,NaN,7.28,8.25,NaN,NaN,NaN,NaN,NaN,NaN,8.93,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.03,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,NaN,NaN,NaN,NaN,2.91,3.93,6.75,6.60,NaN,6.65,-6.12,NaN,7.57,6.21,6.65,NaN,-8.30,7.18,2.82,...,0.00,-3.69,NaN,NaN,7.82,0.24,NaN,7.28,-2.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.63,NaN,NaN,-2.33,NaN,NaN,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,2.48,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,NaN,NaN,NaN,NaN,NaN,NaN,4.13,NaN,NaN,NaN,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,-0.53,5.63,-7.14,-4.08,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46,5.63,-0.92,-2.91,-4.17,2.82,3.40,8.64,6.84,6.80,-0.87,7.38,-3.50,8.88,7.43,5.39,2.23,-0.68,3.40,-0.58,4.42,...,8.59,3.45,0.87,9.27,-4.66,5.73,-0.49,8.35,1.94,5.00,-9.66,8.98,8.98,-9.81,9.13,9.08,9.08,3.98,0.73,9.03,8.98,9.22,8.93,9.13,9.27,-1.99,-9.95,-9.90,9.13,8.83,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,8.88,-8.69,-7.48,-8.83,-1.75,6.60,3.54,1.50,7.67,-0.44,9.22,8.74,9.03,9.08,8.93,3.74,3.20,-9.17,-8.98,8.79,-7.67,-3.06,9.13,8.40,-0.63,-7.18,0.58,8.88,9.27,8.50,...,2.77,8.11,-7.96,8.93,-0.87,-5.87,8.88,-1.12,-8.74,8.74,NaN,NaN,NaN,NaN,NaN,4.90,NaN,NaN,NaN,NaN,-0.29,0.92,-0.78,0.15,-0.10,0.00,-0.19,-0.87,-1.36,-0.58,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,NaN,NaN,NaN,NaN,-7.77,NaN,6.70,-6.75,NaN,NaN,NaN,NaN,-6.46,-1.65,-6.80,-6.41,-6.99,7.23,6.75,-6.99,6.55,NaN,NaN,NaN,NaN,0.49,-0.53,-6.94,-0.49,NaN,6.46,-0.53,NaN,NaN,-7.86,-0.34,NaN,-6.94,NaN,NaN,...,0.49,-0.24,NaN,NaN,-3.11,-6.65,NaN,-0.58,6.31,NaN,NaN,-7.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24981,NaN,NaN,NaN,NaN,-9.71,NaN,4.56,-8.30,NaN,NaN,NaN,NaN,-9.47,NaN,3.45,-0.92,-4.51,-4.13,-5.73,-9.51,2.82,NaN,NaN,NaN,NaN,-0.49,2.91,2.62,8.30,NaN,3.06,5.44,NaN,NaN,-0.68,2.04,NaN,NaN,1.

In [232]:
max_value = test.max()
print('Maximum value in each column : ')
print(max_value)

Maximum value in each column : 
1      9.37
2      9.37
3      9.37
4      9.37
5      9.37
       ... 
96     9.37
97     9.37
98     9.37
99     9.37
100    9.37
Length: 100, dtype: float64


In [ ]:
min_value = test.min()
print('Minimum value in each column : ')
print(min_value)

Minimum value in each column : 
1     -9.95
2     -9.95
3     -9.95
4     -9.95
5     -9.95
       ... 
96    -9.95
97    -9.95
98    -9.95
99    -9.95
100   -9.95
Length: 100, dtype: float64


In [233]:
new_df = test.mean(axis=0, skipna=True)

In [234]:
new_df

1      0.904573
2      0.208460
3      0.316574
4     -1.449114
5      0.403556
         ...   
96     1.528003
97     1.673750
98     0.767692
99    -0.031244
100    1.354711
Length: 100, dtype: float64

In [235]:
new_df.idxmin()

58

In [236]:
new_df.min()

-3.8338796373689923

In [237]:
new_df.idxmax()

50

In [238]:
new_df.max()

3.6650848950824937

In [239]:
low = test[33]
low.iloc[12215]

8.4

In [240]:
arr = test.values
print(arr.shape)

(24983, 100)


In [241]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

2498300 (2498300,)


In [242]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [243]:
new_arr, idx = replace(arr, 0.1)

In [244]:
arr[idx[0][0], idx[1][0]]

2.09

In [245]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [246]:
n_latent_factors = 2

user_ratings = test.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [247]:
latent_item_features

array([[0.74803799, 0.28519648],
       [0.91092306, 0.54562993],
       [0.13076067, 0.99778934],
       [0.25324064, 0.03385297],
       [0.40377735, 0.33095199],
       [0.77917106, 0.62814212],
       [0.21777622, 0.75101955],
       [0.86261441, 0.30599746],
       [0.86178223, 0.93821375],
       [0.33882551, 0.7563249 ],
       [0.36143422, 0.90960154],
       [0.69534412, 0.54919489],
       [0.23940537, 0.0179419 ],
       [0.08090349, 0.43128636],
       [0.53472299, 0.35476048],
       [0.38000496, 0.07464567],
       [0.82508887, 0.60015225],
       [0.63738121, 0.57529157],
       [0.70492386, 0.87505345],
       [0.74454331, 0.61523149],
       [0.24891623, 0.92452006],
       [0.4814712 , 0.76940927],
       [0.09712911, 0.49243872],
       [0.77030681, 0.00778683],
       [0.96394196, 0.18104038],
       [0.05986833, 0.98874617],
       [0.60273195, 0.13046298],
       [0.83501026, 0.59250088],
       [0.27138629, 0.89467039],
       [0.02896734, 0.41267757],
       [0.

In [248]:
latent_user_preferences

array([[0.99015104, 0.08269612],
       [0.08650391, 0.99929999],
       [0.30832008, 0.77098441],
       ...,
       [0.88593268, 0.92358794],
       [0.24718752, 0.71177504],
       [0.34119215, 0.26759975]])

In [249]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id, item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        #if (iteration % 100) == 0:
        print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
        mse_history.append(mse)
    return mse_history      

In [ ]:
#sgd(2)

In [ ]:
num_iter = 20
hist = sgd(num_iter)  # Note how the MSE decreases with the number of iterations
#plt.figure()
#plt.plot(np.arange(0, num_iter, 10), hist)
#plt.xlabel("Iterations")
#plt.ylabel("MSE")
#plt.show()


Iteration 0/20:	MSE=26.205003
Iteration 1/20:	MSE=25.246806
Iteration 2/20:	MSE=24.954144
Iteration 3/20:	MSE=24.718394
Iteration 4/20:	MSE=24.486006
Iteration 5/20:	MSE=24.250867
Iteration 6/20:	MSE=24.013082
Iteration 7/20:	MSE=23.774038
Iteration 8/20:	MSE=23.535439
Iteration 9/20:	MSE=23.299003
Iteration 10/20:	MSE=23.066308
Iteration 11/20:	MSE=22.838671
Iteration 12/20:	MSE=22.617048
Iteration 13/20:	MSE=22.401941
Iteration 14/20:	MSE=22.193308
Iteration 15/20:	MSE=21.990473
Iteration 16/20:	MSE=21.792041
Iteration 17/20:	MSE=21.595817
Iteration 18/20:	MSE=21.398748
Iteration 19/20:	MSE=21.196938


In [250]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[0.76425524, 0.94707289, 0.21198612, ..., 0.45839277, 0.63174695,
        0.4448702 ],
       [0.34970505, 0.62404639, 1.00840218, ..., 0.22438629, 0.25162548,
        0.21886601],
       [0.45051718, 0.70152804, 0.80959617, ..., 0.28121062, 0.34427586,
        0.27374055],
       ...,
       [0.92611534, 1.31095373, 1.03739135, ..., 0.56800943, 0.73347448,
        0.55219442],
       [0.38790139, 0.61353458, 0.74252395, ..., 0.24285607, 0.29455874,
        0.23645752],
       [0.3315432 , 0.45681023, 0.31162269, ..., 0.20238398, 0.26503427,
        0.19667856]])

In [251]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = test.columns

In [252]:
comparison_data
# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, 0.7642552398290778)","(8.79, 0.947072890943941)","(-9.66, 0.21198611822537253)","(-8.16, 0.2535459911632)","(-7.52, 0.42716901056738915)","(-8.5, 0.8234419531024262)","(-9.85, 0.277737749262291)","(4.17, 0.8794233605385441)","(-8.98, 0.9308812052083881)","(-4.76, 0.3980335630153645)","(-8.5, 0.43309498940932567)","(-6.75, 0.7339119847391133)","(-7.18, 0.23853120037067146)","(8.45, 0.11577237987739568)","(-7.18, 0.5587938387555231)","(-7.52, 0.38243520963409533)","(-7.43, 0.8665928623439938)","(-9.81, 0.6786780462987545)","(-9.85, 0.770344616380341)","(-9.85, 0.7880875871913683)","(-9.37, 0.3229188827336728)","(1.5, 0.5403563674022266)","(-4.37, 0.13689526010805325)","(-9.81, 0.7633640302125598)","(-8.5, 0.9694194762154902)","(1.12, 0.14104415595694136)","(7.82, 0.6075844511740607)","(2.86, 0.87578379714135)","(9.13, 0.34269918993461296)","(-7.43, 0.06280887258554049)","(2.14, 0.9703393072648768)","(-4.08, 0.5328405867138941)","(-9.08, 0.12742122579347756)","(7.82, 0.13197051675017316)","(5.05, 0.40730214656031677)","(4.95, 0.2614279206086037)","(-9.17, 0.3217082826380777)","(-8.4, 0.09983776812778995)","(-8.4, 1.0533599136800007)","(-8.4, 0.9161523398159435)",...,"(8.59, 0.913980775547453)","(3.59, 0.5039199666359145)","(-6.84, 0.6815907856682104)","(-9.03, 0.6134589517805842)","(2.82, 0.7401683226952519)","(-1.36, 0.38284586535386855)","(-9.08, 0.6662534114542834)","(8.3, 0.6179794550235546)","(5.68, 0.9701103040074874)","(-4.81, 0.9915674176729424)","(nan, 0.6843796267111094)","(nan, 0.7683195798163602)","(nan, 0.6888023909773946)","(nan, 0.7636560585587029)","(nan, 0.7799502636215407)","(nan, 0.8084166982613219)","(nan, 0.23611633911347862)","(-9.42, 0.5254711279476179)","(nan, 0.3908390152775011)","(nan, 0.6096817629148651)","(nan, 0.37438535275127144)","(-7.72, 0.47208156983404975)","(nan, 0.6268695530295056)","(nan, 0.6415079270310945)","(nan, 0.3877307454456557)","(nan, 0.9586209225543845)","(nan, 0.240441665764276)","(nan, 0.21738284259278293)","(nan, 0.32601033508225585)","(nan, 0.3607390673529997)","(2.82, 0.6019279391850391)","(nan, 0.14632330328423931)","(nan, 0.38418785485647367)","(nan, 0.9347990797747072)","(nan, 0.7899965323748058)","(nan, 0.7078225631628047)","(-5.63, 0.7352051134509948)","(nan, 0.4583927742597174)","(nan, 0.6317469459467024)","(nan, 0.44487019608650324)"
1,"(4.08, 0.34970505338176455)","(-0.29, 0.6240463906460613)","(6.36, 1.0084021848866722)","(4.37, 0.055735576507898114)","(-2.38, 0.3656486404388799)","(-9.66, 0.69510376102173)","(-0.73, 0.7693323220336196)","(-5.34, 0.3804027816867211)","(8.88, 1.0121045240499784)","(9.22, 0.7851051931820862)","(6.75, 0.9402302888398025)","(8.64, 0.6089604358144485)","(4.42, 0.03863884486940705)","(7.43, 0.43798291999758987)","(4.56, 0.4007677725754254)","(-0.97, 0.10746533313511977)","(4.66, 0.6711055558770508)","(-0.68, 0.6300248336505696)","(3.3, 0.9354195767960836)","(-1.21, 0.6792067280507136)","(0.87, 0.9454051173200501)","(8.64, 0.8105198138533006)","(8.35, 0.50049605909109)","(9.17, 0.07441593086635927)","(0.05, 0.2642984012669791)","(7.57, 0.9932328791694277)","(4.71, 0.18251032659697708)","(0.87, 0.6643177779189537)","(-0.39, 0.9175200908815582)","(6.99, 0.41489448349360575)","(6.5, 0.6741974085362115)","(-0.92, 0.07919303033694587)","(7.14, 0.3214492368980608)","(9.03, 0.42320453809576813)","(-1.8, 0.9991499345679825)","(0.73, 0.3699791313128144)","(7.09, 0.8678722446884012)","(3.4, 0.9240648629772751)","(-0.87, 1.083884985120922)","(7.91, 0.8638477511515923)",...,"(-6.7, 0.3383614070965873)","(-3.35, 0.3698551601959433)","(-9.03, 0.7994824781319988)","(4.47, 0.11202674136859524)","(4.08, 0.3278628058121035)","(-3.83, 0.72228518308193)","(8.74, 0.9683926435909788)","(1.12, 0.5858636560408114)","(0.78, 

In [253]:
predict_rating(12435, 55)

0.1747607998110263

In [ ]:
# Load movies data
item_features_df = pd.read_excel('/content/movies_latent_factors.xlsx', sheet_name='Items')
item_features_df.head()

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566


In [ ]:
# Load movies data
user_features_df = pd.read_excel('/content/movies_latent_factors.xlsx', sheet_name='Users')
user_features_df.head()

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,4768,-0.204024,0.161079,-0.090447,0.138495,-0.162934,0.163894,0.051502,-0.088582,0.126829,0.065967,0.085008,0.355404,0.007108,-0.118663,-0.039125
1,156,-0.189652,-0.178979,-0.091490,-0.000823,-0.032646,0.177209,-0.098123,-0.068283,-0.011575,0.120866,-0.009931,-0.048606,0.045916,0.113671,0.179873
2,5323,-0.115308,-0.090886,-0.053129,0.018472,-0.068081,-0.004828,0.113005,0.102107,0.034758,0.000693,-0.073712,-0.019460,0.108372,0.054471,-0.109552
3,174,-0.227462,-0.272532,-0.017231,0.054324,0.214755,-0.072639,-0.033122,-0.086508,-0.131479,0.180403,0.095890,-0.082396,0.036767,-0.165438,0.050692
4,4529,-0.014616,-0.102218,-0.107935,0.155784,-0.123362,-0.118228,-0.013549,-0.050622,0.058698,-0.159600,-0.142382,-0.132836,-0.039897,0.129063,0.102669


In [ ]:
user_features_df.loc[[3], ['Factor15']].head()

,Factor15
3,0.050692


In [254]:
item_features_df.iloc[df1['Factor10'].idxmin()]


Movie ID               597
Title       Titanic (1997)
Factor1            1.56787
Factor2           -1.51695
Factor3           -2.52338
Factor4            1.20448
Factor5            1.72954
Factor6         -0.0889414
Factor7           0.102822
Factor8            1.74779
Factor9            1.81696
Factor10          -2.98681
Factor11         -0.589441
Factor12          -2.56858
Factor13          0.370387
Factor14           2.21593
Factor15          0.456127
key                      0
item_id                 47
Name: 47, dtype: object

In [255]:
n_latent_factors = 15

# Initialise as random values
latent_user_preferences = np.random.random((user_features_df.shape[0], n_latent_factors))
latent_item_features = np.random.random((item_features_df.shape[1], n_latent_factors))

In [256]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[4.74506228, 4.0097732 , 3.98492251, 3.82876086, 4.51231755,
        3.57212879, 4.04600222, 4.80145771, 5.60823088, 2.71747838,
        5.06059626, 4.23567278, 3.10651708, 5.34607167, 2.79313855,
        4.65641244, 4.31523439, 3.68079063, 3.72233449],
       [4.73616599, 4.51780389, 3.21695186, 4.16303745, 5.02292542,
        4.15178786, 3.4703416 , 4.80203969, 5.19133934, 2.61434471,
        4.37915633, 4.76127032, 3.18297787, 5.43414976, 3.13081307,
        4.45068399, 4.40330942, 3.05030664, 3.24857278],
       [2.93782295, 2.46929121, 2.45799875, 2.07535583, 3.18032109,
        2.388481  , 1.93397029, 2.51641665, 3.225503  , 2.20390472,
        3.04557878, 2.29589639, 1.78712723, 3.05451571, 1.90325823,
        2.62892907, 2.65149656, 2.59470573, 2.55215575],
       [3.04976653, 2.94108789, 2.710998  , 2.68764782, 3.08166819,
        2.95029983, 2.23564614, 3.2096391 , 4.10877604, 2.12185316,
        3.31224908, 3.37055565, 1.85781926, 3.75991991, 2.00315657,
        3.081

In [ ]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = user_features_df.columns

In [ ]:
predict_rating(5338, 55)

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15,key,user_id
0,"(4768.0, 3.3658904299365635)","(-0.20402409134103067, 4.647770449399241)","(0.16107871412498165, 3.771347375104536)","(-0.09044749774583173, 4.346277842471619)","(0.13849453099347978, 2.9946850952816653)","(-0.16293365936821244, 3.4394650066246455)","(0.16389439718330012, 5.757997407270375)","(0.05150182396917614, 4.69985372373556)","(-0.08858221522780504, 4.045004782944878)","(0.1268290015160264, 4.6216266627590326)","(0.06596709167792869, 3.7539285120022763)","(0.08500812151118044, 4.174772526766993)","(0.3554044809443143, 5.778906042599685)","(0.007107998359761103, 5.173531425474578)","(-0.11866339698079703, 3.023438171803674)","(-0.039125356639852116, 4.584607052539816)","(0.0, 5.571811427467546)","(0.0, 4.63232473798693)"
1,"(156.0, 3.9641629752898937)","(-0.18965158321975575, 4.4786176169857566)","(-0.17897854162470342, 3.3527926142502866)","(-0.09148956649797665, 3.83361428837261)","(-0.0008227803951894185, 3.3274322042100484)","(-0.03264592333958674, 3.6892431445669827)","(0.1772091693532976, 5.022469500144183)","(-0.09812294163679827, 4.816730691509502)","(-0.06828260805092208, 5.039379482129481)","(-0.011574887128608363, 2.7682625887483194)","(0.12086640092519345, 3.3227166935388)","(-0.00993142958848058, 3.3401645038667103)","(-0.048606208128906606, 4.1532260257916)","(0.04591617005365652, 5.1400659233786605)","(0.1136707144906873, 2.4501856491777563)","(0.17987322302614434, 3.35779736250574)","(0.0, 4.273576618775342)","(1.0, 4.695738307213197)"
2,"(5323.0, 3.318296069295528)","(-0.11530781456056037, 4.701177133446808)","(-0.09088609688252501, 3.423907245032429)","(-0.05312851155182185, 5.251506132546559)","(0.018471580623338233, 3.30329140676762)","(-0.06808075433110529, 2.7962733740419257)","(-0.004828365235235946, 5.183207049635581)","(0.11300466865547716, 5.239029926034301)","(0.10210670647885187, 4.9032809880347346)","(0.03475766536434373, 4.353119174823983)","(0.0006925859083894796, 3.531111110419147)","(-0.07371183943192554, 4.220130569049545)","(-0.019459941050823902, 4.7451757585406655)","(0.10837247723335505, 4.6625675992109175)","(0.05447061984607394, 3.4590893206224744)","(-0.1095522177420274, 3.362353928616048)","(0.0, 4.432715380702671)","(2.0, 4.809682738608731)"
3,"(174.0, 3.5267292984015746)","(-0.22746168114659193, 3.790824447739864)","(-0.27253192676983473, 2.5245657672204547)","(-0.01723089051313282, 4.101169112342037)","(0.05432417888090413, 2.8966223140246994)","(0.21475532181331364, 3.3011665395507026)","(-0.07263876338005151, 4.563115145295715)","(-0.03312155876874437, 4.089518615606549)","(-0.0865081816106039, 4.075618064459567)","(-0.13147905486172737, 3.1484245400875435)","(0.18040285467650258, 3.052995852205882)","(0.09588958698235482, 3.3309752296142308)","(-0.08239632358685363, 4.248872971969634)","(0.03676716974161732, 4.2161003667920225)","(-0.16543839172737043, 2.641028294262688)","(0.05069195040869506, 3.498129758975321)","(0.0, 3.9457364366036134)","(3.0, 4.476501475717788)"
4,"(4529.0, 2.6229189410675526)","(-0.014616406132805185, 3.4241695051569514)","(-0.10221824280347178, 2.7800761651359496)","(-0.10793534556017258, 3.0792105236503153)","(0.1557842607358043, 2.3765474627942678)","(-0.12336155262546537, 2.540702869824614)","(-0.11822813772115061, 4.437666808352667)","(-0.01354853770386976, 3.6592865157292405)","(-0.05062164483498564, 3.3247193055775055)","(0.058697664707379214, 2.9553913091116306)","(-0.1596003646763964, 2.5799133929121263)","(-0.14238174602114692, 2.89974243256017)","(-0.13283562619016845, 3.7288067266707103)","(-0.03989697593739719, 4.092870549534367)","(0.12906339158462574, 1.9593386954554055)","(0.10266922474388747, 2.919066770575787)","(0.0, 4.131344122567328)","(4.0, 3.4253431003815793)"
5,"(783.0, 2.6774245239705925)","(-0.02030071997271344, 2.685230552754472)","(-0.03191881515524646, 2.6326951736341924)","(-0